This document is a designed to be a step by step guide to create a reproduceable basic machine learning model on a STM (IoT) Discovery Kit that can accurately perform gesture recognition. More documentation on the ideas touched on in this document can be found here: https://docs.edgeimpulse.com/docs/tutorials/end-to-end-tutorials/continuous-motion-recognition

This document will contain executable code sections to effectively accomplish the tasks set out above.

PREREQUISITES:
- A STM32F407 Discovery Kit from STMicroelectronics
- A Windows 10, MacOS or Linux 64-bit computer
- USB to Mini-B Cable
- Python3.XX
- Edge Impulse Account

STEPS:
1. Connect the development board to your computer
Use a micro-USB cable to connect the development board to your computer.

2. Download and install the STM32CubeProgrammer:
    2i. Go to this link and download the correct file for your OS: https://www.st.com/en/development-tools/stm32cubeprog.html#get-software
    
    2ii. Unzip the file and run the executable located inside to begin the Setup Wizard

3. Create Mbed OS application
    3i. Download the Mbed OS platform: 
        Windows: https://github.com/ARMmbed/mbed-cli-windows-installer/releases/download/v0.4.10/Mbed_installer_v0.4.10.exe
        MacOS: https://github.com/ARMmbed/mbed-cli-osx-installer/releases/download/v0.0.10/mbed-cli-v0.0.10.dmg
        Linux Instructions: https://os.mbed.com/docs/mbed-os/v6.16/build-tools/install-and-set-up.html

    3ii. Create and navigate to your working directory, if you haven't done so already.

    3iii. Run the command: 
        mbed new {project-name}
        Replacing project name with the name of the project in Edge Impulse.
    
    3iv. Navigate to the newly created /{project-name}/mbed-os folder, and run the command:
        pip install -r requirements.txt

    3v. Run the command: 
        mbed target DISCO_F407ZG
        To set the target to the correct board.
    
    3vi. Move out to the /{project-name} folder, then create a folder with the name of /source. Move into this /source folder, then create a main.cpp file with the following contents:
#include "mbed.h"
#include "stm32l475e_iot01_accelero.h"

static int64_t sampling_freq = 104; // in Hz.
static int64_t time_between_samples_us = (1000000 / (sampling_freq - 1));

// set baud rate of serial port to 115200
static BufferedSerial serial_port(USBTX, USBRX, 115200);
FileHandle *mbed::mbed_override_console(int fd) {
    return &serial_port;
}

int main()
{
    int16_t pDataXYZ[3] = {0};
    Timer t;
    t.start();

    BSP_ACCELERO_Init();

    while(1) {
        int64_t next_tick = t.read_us() + time_between_samples_us;

        BSP_ACCELERO_AccGetXYZ(pDataXYZ);
        printf("%d\t%d\t%d\n", pDataXYZ[0], pDataXYZ[1], pDataXYZ[2]);

        while (t.read_us() < next_tick) {
            /* busy loop */
        }
    }
}

    3vii. Create a new file in the source folder with the file name of stm32l475e_iot01_accelero.h, and with these contents:
/**
  ******************************************************************************
  * @file    stm32l475e_iot01_accelero.h
  * @author  MCD Application Team
  * @brief   This file provides a set of functions needed to manage the accelerometer sensor
  ******************************************************************************
  * @attention
  *
  * Copyright (c) 2017 STMicroelectronics.
  * All rights reserved.
  *
  * This software is licensed under terms that can be found in the LICENSE file
  * in the root directory of this software component.
  * If no LICENSE file comes with this software, it is provided AS-IS.
  *
  ******************************************************************************
  */

/* Define to prevent recursive inclusion -------------------------------------*/
#ifndef __STM32L475E_IOT01_ACCELERO_H
#define __STM32L475E_IOT01_ACCELERO_H

#ifdef __cplusplus
 extern "C" {
#endif

/* Includes ------------------------------------------------------------------*/
#include "stm32l475e_iot01.h"
/* Include Accelero component driver */
#include "../Components/lsm6dsl/lsm6dsl.h"
   
/** @addtogroup BSP
  * @{
  */

/** @addtogroup STM32L475E_IOT01
  * @{
  */
      
/** @addtogroup STM32L475E_IOT01_ACCELERO
  * @{
  */ 
   
/** @defgroup STM32L475_DISCOVERY_ACCELERO_Exported_Types ACCELERO Exported Types
  * @{
  */
typedef enum 
{
  ACCELERO_OK = 0,
  ACCELERO_ERROR = 1,
  ACCELERO_TIMEOUT = 2
} 
ACCELERO_StatusTypeDef;

/**
  * @}
  */

/** @defgroup STM32L475E_IOT01_ACCELERO_Exported_Functions ACCELERO Exported Functions
  * @{
  */
/* Sensor Configuration Functions */
ACCELERO_StatusTypeDef BSP_ACCELERO_Init(void);
void BSP_ACCELERO_DeInit(void);
void BSP_ACCELERO_LowPower(uint16_t status); /* 0 Means Disable Low Power Mode, otherwise Low Power Mode is enabled */
void BSP_ACCELERO_AccGetXYZ(int16_t *pDataXYZ);
/**
  * @}
  */

/**
  * @}
  */

/**
  * @}
  */

/**
  * @}
  */
#ifdef __cplusplus
}
#endif

#endif /* __STM32L475E_IOT01_ACCELERO_H */

    3vii. Compile the code through this command:
        mbed compile -t GCC_ARM --flash


4. Find the Device's Serial Port
Open Device Manager (Windows) or use dmesg | grep tty (Linux) to find the serial port the Discovery Kit is connected to. The device will show up as a COM port on Windows or a /dev/ttyUSBx or /dev/ttyACMx port on Linux.

4. Open a Terminal Emulator
Open a terminal emulator like PuTTY, Tera Term, or minicom.
Configure the terminal to connect to the identified serial port:
Baud rate: 115200 (This value can be configured depending on the setup used)
Data bits: 8
Parity: None
Stop bits: 1
Flow control: None

5. Connect to the board through Terminal Emulator
In this example, PuTTY will be used:
    In the PuTTY Configuration window:
    Under Connection type, select Serial.

    In the Serial line field, enter the COM port of your device (e.g., COM3 on Windows or /dev/ttyACM0 on Linux) found in step 3.
    Set the Speed (baud rate) to 115200 (or whatever baud rate you previously configured).

